In [59]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings("ignore")

fase = '06_evaluar_prediccion'

In [60]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [61]:
folder = gen_config['folder']

path_prod_data = gen_config['path_prod_data']
path_pred_test = gen_config['path_pred_test']
path_pred_futuro = gen_config['path_pred_futuro']

path_kaggle = gen_config['path_kaggle']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
---------------------------------------06_evaluar_prediccion----------------------------------------


In [62]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [63]:
prod_data = pd.read_parquet(f"{folder}/{path_prod_data}")
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
df_test = pd.read_parquet(f"{folder}/{path_pred_test}")
df_fut = pd.read_parquet(f"{folder}/{path_pred_futuro}")

In [64]:
df_pred_test = df_test[['product_id', 'tn_futuro_real', 'tn_prediccion_real']]
df_pred_test['error'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])/df_pred_test['tn_futuro_real']*100
df_pred_test['error_abs'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])
df_pred_test['weight'] = df_pred_test['tn_futuro_real']/df_pred_test['tn_futuro_real'].sum()
df_pred_test.sort_values('weight', ascending=False, inplace=True)

perdida = perdida(df_pred_test['tn_futuro_real'], df_pred_test['tn_prediccion_real'])
print(f"Perdida en test (sin desagrupar): {perdida}")
print(f"Primeras filas de test:\n{df_pred_test.iloc[:6,:4]}")

Perdida en test (sin desagrupar): 0.09545155043846522
Primeras filas de test:
    product_id  tn_futuro_real  tn_prediccion_real      error
0        20001      1504.68856         1423.915240  -5.368109
1        20002      1087.30855         1044.707786  -3.918001
2        20003       892.50129          778.299081 -12.795747
3        20004       637.90002          608.696496  -4.578072
4        20005       593.24443          541.165184  -8.778716
31       20032       527.79811          538.313041   1.992226


In [65]:
# df_pred_ungroup = df_pred_test[['group_index', 'tn_prediccion_real']]
# df_pred_ungroup = df_pred_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_pred_ungroup = df_pred_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_pred_ungroup = df_pred_ungroup[df_pred_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_pred_ungroup['prediccion_final'] = df_pred_ungroup['tn_prediccion_real'] * df_pred_ungroup['proporcion']
# df_pred_ungroup.sort_values('product_id', ascending=True, inplace=True)

# df_pred_ungroup[df_pred_ungroup['group_index'] == 281]

In [66]:
df_pred_fut = df_fut[['product_id', 'tn_futuro_real', 'tn_prediccion_real']]
#df_pred_fut['error'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])/df_pred_fut['tn_futuro_real']*100
#df_pred_fut['error_abs'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])
#df_pred_fut['weight'] = df_pred_fut['tn_futuro_real']/df_pred_fut['tn_futuro_real'].sum()
#df_pred_fut.sort_values('weight', ascending=False, inplace=True)

print(f"Primeras filas de futuro:\n{df_pred_fut.iloc[:6,:4]}")

Primeras filas de futuro:
   product_id  tn_futuro_real  tn_prediccion_real
0       20001             NaN         1280.780584
1       20002             NaN          918.199222
2       20003             NaN          681.608779
3       20004             NaN          482.581765
4       20005             NaN          548.094356
5       20006             NaN          508.729179


In [27]:
# df_fut_ungroup = df_fut[['group_index', 'tn_prediccion_real']]
# df_fut_ungroup = df_fut_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_fut_ungroup = df_fut_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_fut_ungroup = df_fut_ungroup[df_fut_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_fut_ungroup['prediccion_final'] = df_fut_ungroup['tn_prediccion_real'] * df_fut_ungroup['proporcion']
# df_fut_ungroup.sort_values('product_id', ascending=True, inplace=True)

In [67]:
kaggle =  df_pred_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
#kaggle.to_csv('kaggle-new.csv', index=False)
kaggle.to_csv(f"{folder}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{path_kaggle}", index=False)


In [68]:
#estado_control = f"06_evaluar_prediccion Terminado - {nombrefile} - {datetime.now()}"

In [69]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

---------------------------------------06_evaluar_prediccion----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------

